## Imports

In [1]:
import pandas as pd
import numpy as np
from typing import Tuple

## Sum of squares

In [11]:
def sum_of_squares(y: pd.Series,w: np.array, ph: pd.DataFrame) -> float:
    """Calculates the sum of squares error function

    Parameters
    ----------
    y : pd.Series
        target variable
    w : np.array
        current weights
    ph : pd.DataFrame
        phi function evaluated for every entry of X

    Returns
    -------
    float
        _description_
    """
    res = 0
    wt = np.array([w])
    ph_np = ph.to_numpy()
    for i,val in y.items():
        ph_i = ph_np[i].reshape(-1,1)
        res = res + 0.5*(val - np.matmul(wt,ph_i))**2
    return res[0][0]

## Min max scaling

In [3]:
def min_max_scaling(ds: pd.Series) -> Tuple[pd.Series, Tuple[float,float]]:
    """Linearly scales a pandas series from min -> max to 0 -> 1

    Parameters
    ----------
    ds : pd.Series
        Series to scale

    Returns
    -------
    Tuple[pd.Series, Tuple[float,float]]
        pd.Series: Scaled Series
        Tuple[float,float]
            float: Minimum of the original series
            float: Maximum of the original series
    """
    mx = np.nanmax(ds)
    mn = np.nanmin(ds)
    delta = mx - mn
    ds_out = (ds - mn)/delta
    return ds_out, (mn,mx)

In [4]:
df = pd.read_csv('../data/1.raw/electricityConsumptionAndProductioction.csv', parse_dates=['DateTime'])
X_cols = ['Nuclear', 'Wind', 'Hydroelectric', 'Oil and Gas', 'Coal', 'Solar', 'Biomass']
y_col = 'Production'

X = df[X_cols]
for c in X.columns:
    X[c] = min_max_scaling(X[c])[0]

y = df[y_col]
y = min_max_scaling(y)[0]

C:\Users\an2fe\AppData\Local\Temp\ipykernel_12488\1746569770.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c] = min_max_scaling(X[c])[0]


In [12]:
bias = 0
bf = 'lineal'
w = np.ones(X.shape[1])
eta = 1.15e-7
ph = X

sum_of_squares(y,w,ph)

136091.66694520967